In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
MODEL="llama2"
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
model= Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

model.invoke("tell me a joke")

C:\Users\Admin\AppData\Local\Temp\ipykernel_2376\3251092675.py:8: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings()


"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing!"

In [20]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
chain = model | parser 
chain.invoke("tell me a joke")

"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you find that amusing!"

In [27]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("career.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'career.pdf', 'page': 0}, page_content='A Guide to \nAdvancing Your \nCareer with Essential \nBusiness Skills'),
 Document(metadata={'source': 'career.pdf', 'page': 1}, page_content='Contents \n3 The Business Skills You \nNeed to Succeed \n7 How to Develop Business \nSkills \n11 How to Advance Your \nCareer with Business Skills \n15 Investing in Yourself and \nYour Career \nIN TODAY’S RAPIDLY CHANGING ECONOMY, there’s a growing \nneed for professionals with the knowledge and skills to contribute to key business \ndiscussions and drive strategic decision-making. \nAccording to the US Bureau of Labor Statistics, the number of business and \nfnancial jobs is projected to grow 10 percent through 2026—faster than the \naverage for all occupations. And research by LinkedIn found that individuals \nskilled in management, leadership, and analytical reasoning are in high demand by \ncompanies across industries. \nNo matter your profession, enhancing your business k

In [28]:
from langchain.prompts import PromptTemplate
template="""
Answer the question based on the context below. If you can't 
answer the question, reply " I don't know".

Context: {context}

Question: {question}
""" 
prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))



Answer the question based on the context below. If you can't 
answer the question, reply " I don't know".

Context: Here is some context

Question: Here is a question



In [29]:
chain = prompt | model | parser 

In [30]:
chain.input_schema.schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [31]:
chain.invoke(
    {
        "context": "Im bruce wayne , and you are alfred . you can call me master wayne or master bruce wayne ",
        "question": "What's my name?"
    }
)

' Ah, a simple question, yet one that is so crucial to our daily lives, Master Wayne. *adjusts monocle* My apologies, I mean Master Bruce Wayne. Your name, sir, is... *peers intensely* Bruce Wayne!'

In [32]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

In [35]:
retriever= vectorstore.as_retriever()
retriever.invoke("Management")

[Document(metadata={'source': 'career.pdf', 'page': 3}, page_content='1. An Understanding of Economics \nA baseline knowledge of economics can be valuable in any industry. In addition to \nan in-depth understanding of pricing strategies \nand demand, studying economics \ncan provide you with a toolkit for making business decisions that give your company \na competitive advantage in the marketplace. \n2. Data Analysis \nResearch by McKinsey shows that an increasing share of frms are using business \nanalytics to generate \ngrowth. Companies, including Microsoft, Uber, and Blue \nApron, leverage data to improve their services and operations. \nKnowing how to summarize datasets, recognize trends, and test hypotheses can \ngive you an analytical framework for approaching complex business problems, \nenabling you to make informed decisions that could beneft your frm. \n“Using data analytics is a very efective way to have infuence in an organization,” \nsays Harvard Business School Professor

In [37]:
from operator import itemgetter
chain =(
    {"context": itemgetter("question") | retriever, "question":itemgetter("question")}
    | prompt 
    | model 
    | parser 
)

chain.invoke({
    "question": "Why is management important?"
})

' Based on the context provided, it seems that management is intrinsically linked to organizational performance and can drive team productivity and morale. According to a recent study by Gallup, managers account for 70 percent of variance in employee engagement, highlighting the need for companies to develop leaders who can drive team productivity and morale. Additionally, research by the Association for Talent Development shows that participating in a mentoring relationship can lead to greater professional development and a deeper understanding of organizational culture. Overall, developing strong managerial skills is important for career advancement and organizational success.'

In [38]:
questions = [
    "How to Advance Your Career with Business Skills?",
    "Why Read Business Articles and Blogs?",
    "why Your Resume is important?",
]
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: How to Advance Your Career with Business Skills?
Answer: Based on the provided context, the answer to the question "How to Advance Your Career with Business Skills?" is:

Developing essential business skills can help you succeed in your career. Here are five ways to advance your career with business skills:

1. Networking: Tap into your professional network and identify a mentor at your organization who can offer advice and guidance on how to grow in your chosen area of expertise.
2. Seek Feedback: Regularly seek feedback from your managers and colleagues to identify your strengths and weaknesses, and work on improving your skills.
3. Develop Essential Skills: Identify the essential business skills you need to develop, such as business analytics or financial accounting, and work on acquiring them through various means, such as online courses, workshops, or professional certifications.
4. Collaborate with Others: Share your knowledge and collaborate with others in the HBS Onli